In [1]:
import torch
import pandas as pd
# from torch.utils.data import DataLoader, SequentialSampler
from sklearn.preprocessing import MaxAbsScaler
import numpy as np
from sklearn.cluster import KMeans
from collections import Counter
from itertools import chain
from tqdm.auto import tqdm
import pickle

from dataset import TwitterDataset
from autoencoders import LinearAutoEncoder

In [2]:
cluster_results = pickle.load(open('cluster_results/kmeans_7_30m.pkl','rb'))

In [5]:
cluster_results[0][0]

{'time_start': Timestamp('2022-11-09 05:53:33.788000'),
 'time_end': Timestamp('2022-11-09 06:23:33.788000'),
 'label': 0,
 'by_distance': array([1522623, 1521168, 1528569, ..., 1526658, 1524362, 1521021]),
 'distance': array([0.60615003, 0.33277526, 0.48974606, ..., 0.5521055 , 0.42956275,
        0.35946658], dtype=float32)}

In [6]:
dset = TwitterDataset(
    'timestamp',
    'spam_idx.pkl',
    sentiment_path='sentiment/vader/',
    # whole_text_path = 'whole_text',
    token_path = 'tokens/text',
    # embedding_path = 'embeddings/all-MiniLM-L6-v2/',
)

loading from timestamp:   0%|          | 0/221 [00:00<?, ?it/s]

loading sentiment..:   0%|          | 0/221 [00:00<?, ?it/s]

loading tokens...:   0%|          | 0/221 [00:00<?, ?it/s]

no spam here!
loaded dataset. took 217.59297010488808 ms. Got 20050096 items


In [14]:


clust_words = []
for one_clus in cluster_results[0]:
    word_counts = {}
    for idx,dis in zip(tqdm(one_clus['by_distance']),one_clus['distance']):
        for t in dset.tokens[idx]:
            if not t in word_counts:
                word_counts[t] = dis
            else:
                word_counts[t] += dis
    by_weight = sorted(word_counts.items(),key=lambda x: x[1],reverse=True)
    clust_words.append(by_weight)



  0%|          | 0/1433 [00:00<?, ?it/s]

  0%|          | 0/4839 [00:00<?, ?it/s]

  0%|          | 0/2556 [00:00<?, ?it/s]

  0%|          | 0/3978 [00:00<?, ?it/s]

  0%|          | 0/569 [00:00<?, ?it/s]

  0%|          | 0/2067 [00:00<?, ?it/s]

  0%|          | 0/707 [00:00<?, ?it/s]

In [17]:
for c in clust_words:
    print([w for w,sco in c[:10]])

['roll', 'security', 'bridge', 'amp', 'like', 'nt', 'people', 'volt', 'slow', 'eat']
['$', 'nft', 'token', 'crypto', 'join', 'binance', 'premium', 'apesport', '+', 'nfts']
['twitter', 'join', 'telegram', 'maybe', 'tweet', 'check', 'follow', 'cryptocurrencies', 'gives', 'constantly']
['$', 'crypto', 'binance', 'btc', 'ftt', 'bnb', 'bitcoin', 'ftx', 'sol', 'project']
['security', 'social', 'amp', 'people', 'sushi', 'french', 'medicare', 'roll', 'republicans', 'defence']
['$', 'talking', 'uniswap', 'exploited', 'dude', '200k', 'far', 'leaked', 'alpha', 'group']
['pump', 'kucoin', 'event', 'signal', 'big', 'happen', 'group', 'wallstreetbets', 'hours', '1']
